In [17]:
import os
import logging
import time
import argparse
import random
from datetime import timedelta

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Bidirectional, Masking
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from scipy.interpolate import CubicSpline

logging.basicConfig(format="%(levelname)s %(message)s", level=logging.INFO)
MASK_VAL = -1.0


In [18]:
# GPU check
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    print("GPU device not found. Đảm bảo bạn đã chọn runtime GPU.")
else:
    print(f"Found GPU at: {device_name}")

Num GPUs Available:  1
Found GPU at: /device:GPU:0


I0000 00:00:1750762546.606573 1727937 gpu_device.cc:2019] Created device /device:GPU:0 with 22126 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:01:00.0, compute capability: 8.6


In [19]:
def load_and_align_data(files: dict, variables: list, mask: float = MASK_VAL):
    idx = None
    for path in files.values():
        df = pd.read_csv(path, parse_dates=['datetime'],
                         index_col='datetime').fillna(mask)
        idx = df.index if idx is None else idx.intersection(df.index)
    data, names = [], []
    for name, path in files.items():
        df = pd.read_csv(path, parse_dates=['datetime'],
                         index_col='datetime').fillna(mask)
        for var in variables:
            if var not in df.columns:
                continue
            data.append(df.loc[idx, var].tolist())
            names.append(f"{name}_{var}")
    return np.array(data), names, idx

def data_normalize(data):
    data = np.array(data)
    mask = data == MASK_VAL
    minv = np.min(data[~mask])
    maxv = np.max(data[~mask])
    norm = (data - minv) / (maxv - minv + 1e-8)
    norm[mask] = MASK_VAL
    return norm, (minv, maxv)

def data_denormalize(data, scale):
    minv, maxv = scale
    return data * (maxv - minv + 1e-8) + minv


In [20]:
def interpolate(data):
    data = np.array(data)
    interp = data.copy()
    for i in range(data.shape[0]):
        x = np.arange(data.shape[1])
        mask = data[i] != MASK_VAL
        if np.sum(mask) < 2:
            continue
        f = CubicSpline(x[mask], data[i][mask])
        interp[i, ~mask] = f(x[~mask])
    return interp

def create_sequences(X, Y, T_in, T_out, offset=0, stride=1):
    X = X.T
    Y = Y.T
    samples, targets = [], []
    for start in range(offset, len(X) - T_in - T_out + 1, stride):
        end = start + T_in
        out_end = end + T_out
        samples.append(X[start:end])
        targets.append(Y[end:out_end])
    return np.array(samples), np.array(targets)


In [29]:
def stacked_LSTM(X, Y, learning_rate=6e-6):
    time_step = X.shape[1]
    input_dim = X.shape[2]
    out = Y.shape[2]
    T_out = Y.shape[1]

    start = time.time()
    model = Sequential()
    model.add(Masking(mask_value=MASK_VAL, input_shape=(time_step, input_dim)))
    model.add(Bidirectional(LSTM(64, activation='elu', return_sequences=True)))
    model.add(Bidirectional(LSTM(32, activation='elu', return_sequences=False)))
    model.add(Dense(T_out * out))
    model.add(tf.keras.layers.Reshape((T_out, out)))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(loss='mean_absolute_error', optimizer=optimizer)

    hist = model.fit(
        X, Y,
        epochs=50,
        validation_split=0.2,
        verbose=1,
        batch_size=10
    )
    model.summary()
    end = time.time()
    print("Total compile time: --------", end - start, 's')
    return model, hist

In [23]:
def compute_target_weights(positions, target_pos, alpha=2.0, eps=1e-6):
    coords = np.array(list(positions.values()))
    dists = np.linalg.norm(coords - np.array(target_pos), axis=1)
    w = 1.0 / (dists**alpha + eps)
    return w / w.sum()


In [ ]:
def leave_one_sensor_out_pipeline(files, variables, sensor_xy,
                                 T_in, T_out, new_xy, alpha=2.0,
                                 offset=0, stride=1, split=0.8,
                                 learning_rate=6e-6,
                                 mask_val=MASK_VAL):
    # 1) Load + align + normalize + interpolate
    data, names, idx = load_and_align_data(files, variables, mask=mask_val)
    norm_data, scale = data_normalize(data)
    interp_data = interpolate(norm_data)

    # 2) Tạo các sequence X, Y
    X, Y = create_sequences(interp_data, interp_data, T_in, T_out, offset, stride)
    n_train = int(len(X) * split)
    X_train, Y_train = X[:n_train], Y[:n_train]
    X_test,  Y_test  = X[n_train:], Y[n_train:]

    preds_list, truths_list = [], []
    scales_list, models, histories = [], [], []

    # 3) Với mỗi sensor held-out
    for sensor, pos in sensor_xy.items():
        hold_cols  = [j for j, nm in enumerate(names) if nm.startswith(f"{sensor}_")]
        train_cols = [j for j in range(len(names)) if j not in hold_cols]

        X_tr = X_train[:, :, train_cols]
        Y_tr = Y_train[:, :, hold_cols]
        X_te = X_test[:, :, train_cols]
        Y_te = Y_test[:, :, hold_cols]

        # 4) Train & predict
        model, hist = stacked_LSTM(X_tr, Y_tr, learning_rate=learning_rate)
        models.append(model)
        histories.append(hist)

        pred_norm   = model.predict(X_te)
        pred_denorm = data_denormalize(pred_norm, scale)
        truth_denorm = data_denormalize(Y_te, scale)

        # 5) Tính metrics độc lập
        y_true = truth_denorm.ravel()
        y_pred = pred_denorm.ravel()
        valid = (~np.isnan(y_true)) & (~np.isnan(y_pred))

        mae  = mean_absolute_error(y_true[valid], y_pred[valid])
        rmse = np.sqrt(mean_squared_error(y_true[valid], y_pred[valid]))
        r2   = r2_score(y_true[valid], y_pred[valid])
        print(f"[{sensor}] MAE={mae:.4f}, RMSE={rmse:.4f}, R2={r2:.4f}")

        preds_list.append(pred_denorm)
        truths_list.append(truth_denorm)
        scales_list.append(scale)

    # 6) Kết hợp dự báo cho vị trí mới bằng trọng số
    w = compute_target_weights(sensor_xy, new_xy, alpha=alpha)
    preds_stack = np.stack([p.squeeze(-1) for p in preds_list], axis=1)
    truths_stack = np.stack([t.squeeze(-1) for t in truths_list], axis=1)

    pred_new = np.tensordot(preds_stack, w, axes=([1], [0]))
    truth_new = np.tensordot(truths_stack, w, axes=([1], [0]))

    # 7) Tính metrics cho vị trí mới
    y_true_n = truth_new.ravel()
    y_pred_n = pred_new.ravel()
    valid_n = (~np.isnan(y_true_n)) & (~np.isnan(y_pred_n))
    mae_n  = mean_absolute_error(y_true_n[valid_n], y_pred_n[valid_n])
    rmse_n = np.sqrt(mean_squared_error(y_true_n[valid_n], y_pred_n[valid_n]))
    r2_n   = r2_score(y_true_n[valid_n], y_pred_n[valid_n])
    print(f"[new_position] MAE={mae_n:.4f}, RMSE={rmse_n:.4f}, R2={r2_n:.4f}")

    return {
        'per_sensor_preds': np.array(preds_list),
        'per_sensor_truths': np.array(truths_list),
        'scales': scales_list,
        'models': models,
        'histories': histories,
        'combined_pred': pred_new,
        'combined_truth': truth_new,
        'combined_metrics': {'MAE': mae_n, 'RMSE': rmse_n, 'R2': r2_n}
    }


In [ ]:
if __name__ == "__main__":
    TRAIN_IN      = 335    # timesteps fed into network
    TEST_OUT      = 30     # timesteps predicted
    PRED_GAP      = 70     # offset between in & out
    STRIDE        = 15
    SPLIT         = 0.8    # train/test split
    NEW_XY        = (0.6, 0.5)   # vị trí mới
    ALPHA         = 2.0
    learning_rate = 6e-6  # chỉnh tay learning rate

    variables = ['PM2_5.ug.m3.']
    base_dir  = '/home/nghia/Desktop/Minh_Workspace/V_IndoorCare/dataset/'
    files     = {f'sensor{i}': f'{base_dir}{i:02d}_01mins.csv' for i in range(1,5)}
    sensor_xy = {
        'sensor1': (0.4,0.2),
        'sensor2': (1.0,0.0),
        'sensor3': (0.5,0.8),
        'sensor4': (0.7,0.3),
    }

    logging.info("Start leave-one-sensor-out training + weighted interpolation...")

    results = leave_one_sensor_out_pipeline(
        files=files,
        variables=variables,
        sensor_xy=sensor_xy,
        T_in=TRAIN_IN,
        T_out=TEST_OUT,
        new_xy=NEW_XY,
        alpha=ALPHA,
        offset=PRED_GAP,
        stride=STRIDE,
        split=SPLIT,
        learning_rate=learning_rate
    )

    # --- Lấy kết quả & in ra ---
    pred_new = results['combined_pred']
    combined_metrics = results['combined_metrics']

    print("First-window prediction at new pos:", pred_new[0])
    print(f"Combined metrics @ {NEW_XY}: {combined_metrics}")


INFO Start leave-one-sensor-out training + weighted interpolation...


Epoch 1/5


/home/nghia/Desktop/Minh_Workspace/.conda/lib/python3.12/site-packages/keras/src/layers/core/masking.py:48: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


499/499 ━━━━━━━━━━━━━━━━━━━━ 55s 99ms/step - loss: 0.0318 - val_loss: 0.0071
Epoch 2/5
499/499 ━━━━━━━━━━━━━━━━━━━━ 46s 92ms/step - loss: 0.0127 - val_loss: 0.0064
Epoch 3/5
499/499 ━━━━━━━━━━━━━━━━━━━━ 45s 91ms/step - loss: 0.0116 - val_loss: 0.0086
Epoch 4/5
499/499 ━━━━━━━━━━━━━━━━━━━━ 45s 91ms/step - loss: 0.0112 - val_loss: 0.0065
Epoch 5/5
499/499 ━━━━━━━━━━━━━━━━━━━━ 46s 91ms/step - loss: 0.0106 - val_loss: 0.0060


Model: "sequential_5"

...

 Optimizer params: 155,966 (609.25 KB)

Total compile time: -------- 238.4985430240631 s
49/49 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step


INFO Combined metrics @ (0.6, 0.5): {'MAE': 0.9568685152998433, 'RMSE': np.float64(2.038373300885398), 'R2': 0.981693981542384}
INFO ✓ Done.


[sensor4] MAE=1.3221, RMSE=2.4783, R2=0.9724
[new_position] MAE=0.9569, RMSE=2.0384, R2=0.9817
First-window prediction at new pos: [18.35189657 18.48240507 18.39876022 18.26242483 18.70895793 18.33421652
 18.42604812 18.48496426 18.34086546 18.27777544 18.36456573 18.30772474
 18.43639242 18.55187077 18.588383   18.33116564 18.25780088 18.37434193
 18.63148703 18.28848784 18.20562228 18.30372737 18.33850992 18.27659931
 18.26339585 18.35321925 18.40870068 18.35978248 18.23656966 18.17202903]


: 